In [1]:
import numpy as np
from ngsolve import *
from netgen.occ import *
from netgen.meshing import MeshingParameters

# --- 1. Define Parameters and Materials ---

# Physical Constants
freq = 5e9  # 5 GHz
c0 = 299792458.0  # speed of light in vacuum
mu0 = 4 * np.pi * 1e-7
eps0 = 1 / (mu0 * c0**2)
omega = 2 * np.pi * freq
k0 = omega / c0

# Material Properties
mu_r_air = 1.0
mu_r_copper = 1.0
sigma_copper = 5.8e7  # S/m
eps_r_copper = 1 - 1j * sigma_copper / (omega * eps0)
eps_r_air = 1.0

# Grating dimensions
base_x = 0.05  # 50 mm
base_y = 0.001 # 1 mm
base_z = 0.001 # 1 mm
rect_x = 0.005 # 5 mm
rect_y = 0.01  # 10 mm
rect_z = 0.01  # 10 mm
spacing = 0.012 # 12 mm
num_elements = 5

# Domain and PML dimensions (chosen to be large enough)
pml_thickness = 0.02
total_x = 0.1
total_y = 0.02
total_z = 0.05

# --- 2. Create Geometry ---
print("creating geometry") 
# Full domain including PML
full_domain = Box(Pnt(-total_x/2, -total_y/2, -total_z/2), Pnt(total_x/2, total_y/2, total_z/2))

# Inner box (non-PML region)
inner_domain = Box(Pnt(-total_x/2 + pml_thickness, -total_y/2 + pml_thickness, -total_z/2 + pml_thickness),
                   Pnt(total_x/2 - pml_thickness, total_y/2 - pml_thickness, total_z/2 - pml_thickness))

# Define the PML region as a volume object
pml_region = full_domain - inner_domain
pml_region.mat("pml")

grating_shapes = []

# Add the base plate to the list
base = Box(Pnt(-base_x/2, -base_y/2, 0), Pnt(base_x/2, base_y/2, base_z))
grating_shapes.append(base)

start_x = -((num_elements - 1) * spacing) / 2
for i in range(num_elements):
    current_x = start_x + i * spacing
    # Use a translation to place the grating element at the correct x position
    # The fix is here: change Pnt to Vec
    grating_element = Box(Pnt(-rect_x/2, -rect_y/2, base_z), Pnt(rect_x/2, rect_y/2, base_z + rect_z)).Move(Vec(current_x, 0, 0))
    grating_shapes.append(grating_element)

# Perform a single Glue operation (union) on the list
grating_region = Glue(grating_shapes)
grating_region.mat("copper")

# Subtract the grating from the inner air domain
air_region = inner_domain - grating_region
air_region.mat("air")

# Glue all distinct material regions together
geo = OCCGeometry(Glue([air_region, grating_region,pml_region]))

# --- 3. Generate Mesh and Set PML ---
mp = MeshingParameters(maxh=0.002)
ngmesh = geo.GenerateMesh(mp=mp)
mesh = Mesh(ngmesh)
print("mesh generated") 

creating geometry
mesh generated


In [2]:

# Now, with the PML region explicitly defined, we can set the PML transformation
mesh.SetPML(pml.Cartesian(mins = (-total_x/2, -total_y/2, -total_z/2),
                          maxs = (total_x/2, total_y/2, total_z/2),
                          alpha=1j*k0), "pml")

# --- 4. Define Finite Element Space ---
fes = HCurl(mesh, order=2, complex=True)

# --- 5. Define Total Field and Scattered Field ---
E_scat = fes.TrialFunction()
v = fes.TestFunction()
# Incident Plane Wave (E-field polarized in y-direction, propagating in z-direction)
E_inc_coeff = CoefficientFunction( (0, 1.0, 0) )
E_inc = E_inc_coeff * exp(-1j * k0 * z)

# Manually define the curl of the incident field
E_inc_curl = CoefficientFunction( (1j * k0, 0, 0) ) * exp(-1j * k0 * z)

# --- 6. Define Coefficients and Bilinear Form ---
epscf = { "air": eps_r_air, "copper": eps_r_copper, "pml": eps_r_air }
mucf = { "air": mu_r_air, "copper": mu_r_copper, "pml": mu_r_air }

epsilon = mesh.MaterialCF(epscf, default=epscf["air"])
mu = mesh.MaterialCF(mucf, default=mucf["air"])

a = BilinearForm(fes, symmetric=True)
a += (1/mu) * curl(E_scat) * curl(v) * dx
a += -k0**2 * epsilon * E_scat * v * dx

f = LinearForm(fes)
f += (k0**2 * (epsilon - eps_r_air)) * E_inc * v * dx("copper")
f += - (1/mu_r_air) * E_inc_curl * curl(v) * dx("air")
f += - (-k0**2 * eps_r_air) * E_inc * v * dx("air")
f += - (1/mu_r_air) * E_inc_curl * curl(v) * dx("pml")
f += - (-k0**2 * eps_r_air) * E_inc * v * dx("pml")

# --- 7. Solve and Post-process ---
gfu_scat = GridFunction(fes, "E_scat")

# Solve
with TaskManager():
    print("Assembling matrices...")
    a.Assemble()
    f.Assemble()
    print("Solving linear system using an iterative solver...")
    
    # Create a preconditioner for the iterative solver
    preconditioner = a.mat.CreateSmoother(freedofs=fes.FreeDofs())
    
    # Create a Conjugate Gradient (CG) iterative solver
    # The CG solver uses the preconditioner to accelerate convergence
    solver = CGSolver(mat=a.mat, pre=preconditioner, printrates=True, maxsteps=2000)
    
    # Solve the system by multiplying the solver with the linear form vector
    gfu_scat.vec.data = solver * f.vec
    
    print("Solution complete.")

# Compute total field
gfu_total = GridFunction(fes, "E_total")
gfu_total.Set(E_inc + gfu_scat)

# E-field magnitude visualization
E_mag = Norm(gfu_total)
Draw(E_mag,mesh)

Assembling matrices...
Solving linear system using an iterative solver...
Solution complete.


<class 'TypeError'>: Draw(): incompatible function arguments. The following argument types are supported:
    1. (cf: ngsolve.fem.CoefficientFunction, mesh: ngsolve.comp.Mesh, name: str, sd: int = 2, autoscale: bool = True, min: float = 0.0, max: float = 1.0, draw_vol: bool = True, draw_surf: bool = True, reset: bool = False, title: str = '', number_format: str = '%.3e', unit: str = '', **kwargs) -> None
    2. (gf: ngsolve.comp.GridFunction, sd: int = 2, autoscale: bool = True, min: float = 0.0, max: float = 1.0, **kwargs) -> None
    3. (mesh: ngsolve.comp.Mesh, **kwargs) -> None
    4. (arg0: object) -> None

Invoked with: <ngsolve.fem.CoefficientFunction object at 0xc49bba8>, <ngsolve.comp.Mesh object at 0xd5b6358>